# Day 1: The Tyranny of the Rocket Equation

In [1]:
import os
import unittest

In [2]:
filename = os.path.join(os.getcwd(),"d1_input")
with open(filename) as f:
    mass_inputs = [int(mass) for mass in f.readlines()]

In [3]:
def requiredFuel(mass):
    return mass // 3 -2

def totalFuel(mass):
    input_fuel = requiredFuel(mass)
    f = requiredFuel(input_fuel)
    total_fuel = input_fuel
    while f>0:
        total_fuel += f
        f = requiredFuel(f)
    return total_fuel

def fuelSum(mass_inputs,callback):
    return sum(map(callback,mass_inputs))

In [4]:
# Answer 1
fuelSum(mass_inputs,requiredFuel)

3405637

In [5]:
# Answer 2
fuelSum(mass_inputs,totalFuel)

5105597

# Day 2: 1202 Program Alarm

In [54]:
def run_program(intcode_program):
    for i in range(0,len(intcode_program),4):
        if intcode_program[i]==1:
            intcode_program[intcode_program[i+3]] = intcode_program[intcode_program[i+1]] + intcode_program[intcode_program[i+2]]
        elif intcode_program[i]==2:
            intcode_program[intcode_program[i+3]] = intcode_program[intcode_program[i+1]] * intcode_program[intcode_program[i+2]]
        elif intcode_program[i]==99:
            break
        else:
            raise ValueError(f"Something went wrong. Opcode is {intcode_program[i]} at index {i} for the current program {intcode_program}")
    return intcode_program


In [55]:
class Test(unittest.TestCase):
    def test(self):
        self.states = [([1,9,10,3,2,3,11,0,99,30,40,50],[3500,9,10,70,2,3,11,0,99,30,40,50]),
                ([1,0,0,0,99],[2,0,0,0,99]),
                ([2,3,0,3,99],[2,3,0,6,99]),
                ([2,4,4,5,99,0],[2,4,4,5,99,9801]),
                ([1,1,1,4,99,5,6,0,99],[30,1,1,4,2,5,6,0,99])]
        for (program,output) in self.states:
            self.assertEqual(run_program(program),output)

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [56]:
filename = os.path.join(os.getcwd(),"d2_input")
with open(filename) as f:
    initial_program = [int(opcode) for opcode in f.read().split(",")]
    intcode_program = initial_program[:]
intcode_program[1]=12
intcode_program[2]=2

# Answer 1
run_program(intcode_program)[0]

False

In [58]:
def findInput(init_program):
    for noun in range(50):
        for verb in range(50):
            program = init_program[:]
            program[1]=noun
            program[2]=verb
            if run_program(program)[0] == 19690720:
                return noun,verb,100*noun+verb
    
    return None

# Answer 2
findInput(initial_program)

(40, 19, 4019)

# Day 3: Crossed Wires

In [2]:
import scipy.sparse as sparse
import numpy as np
from copy import copy,deepcopy
import math

In [3]:
class Point:
    def __init__(self,i,j):
        self.i = i
        self.j = j

    def __str__(self):
        return f"Point({self.i},{self.j})"

    def manhattanDistance(self,other_point):
        return abs(self.j-other_point.j)+abs(self.i-other_point.i)

In [259]:
class Wire:
    def __init__(self,string,size=20,center=None,fixed_size=False):
        self.full_path = [(el[0],int(el[1:])) for el in string.replace("\n","").split(",")]
        self.flag_changed = False
        self.create_path(size=size,fixed_size=fixed_size,center=center)

    def create_path(self,limit_try=10,size=None,center=None,fixed_size=False):
        def fill_mat():
            self.central_port = Point(self.size//2,self.size//2)
            if center:
                self.central_port = center
                self.size = size
                row = [self.central_port.i]
                col = [self.central_port.j]
            else:
                row = [self.size//2]
                col = [self.size//2]
                
            current_point = copy(self.central_port)
            
            for (direction,nb_steps) in self.full_path:
                if direction=="U":
                    t_row = [i for i in range(current_point.i-1,current_point.i-nb_steps-1,-1)]
                    row += t_row
                    col +=[current_point.j]*len(t_row)
                    current_point.i-=nb_steps
                elif direction=="D":
                    t_row = [i for i in range(current_point.i+1,current_point.i+nb_steps+1)]
                    row += t_row
                    col +=[current_point.j]*len(t_row)
                    current_point.i+=nb_steps
                elif direction=="L":
                    t_col = [i for i in range(current_point.j-1,current_point.j-nb_steps-1,-1)]
                    col += t_col
                    row +=[current_point.i]*len(t_col)                    
                    current_point.j-=nb_steps            
                elif direction=="R":
                    t_col = [i for i in range(current_point.j+1,current_point.j+nb_steps+1)]
                    col += t_col
                    row +=[current_point.i]*len(t_col)                       
                    current_point.j+=nb_steps
            
            self.row_id = row
            self.col_id = col
            val = [1]*len(row)
            val[0]=0.5
            if not fixed_size:
                self.size = max(max(row),max(col),size)+1
            self.sparse = sparse.coo_matrix((val,(row,col)),shape=(self.size,self.size)).todok().tocsc()
        
        
        if size is not None:
            self.size = size
        count_try = 0
        while count_try<limit_try:
            try:
                fill_mat()
                print(f"Wire path created of size {self.size}")
                count_try = limit_try
            except ValueError as e:
                self.size = int(self.size*1.5)
                self.flag_changed = True
        if count_try >limit_try:
            raise IndexError("Couldn't create wire path, the matrix was too small")
        
    def wiresCross(self,other_wire):
        self.intersections = self.sparse.multiply(other_wire.sparse)
        return self.intersections
    
    def getIntersectCoord(self,other_wire=None,intersect_coord=None):
        if other_wire:
            self.wiresCross(other_wire)
            self.intersec_coord = coord = np.argwhere(self.intersections>=1)
        elif intersect_coord:
            self.intersec_coord = intersect_coord

    def getClosestIntersectionDist(self,other_wire):
        min_dist = math.inf
        self.getIntersectCoord(other_wire=other_wire)
        coord = self.intersec_coord
        for i in range(len(coord)):
            dist = self.central_port.manhattanDistance(Point(coord[i,0],coord[i,1]))
            if min_dist>dist:
                min_dist = dist
        return min_dist
    
    def isIntersect(self,coord,intersect_array):
        for idx,point in enumerate(intersect_array):
            if coord[0]==point[0] and coord[1]==point[1]:
                return True,idx
        return False,None
    
    def signalAtIntersect(self,other_wire=None,intersect_coord=None):
        self.getIntersectCoord(other_wire=other_wire,intersect_coord=intersect_coord)
        self.intersect_length=[None]*len(self.intersec_coord)
        for idx,(i,j) in enumerate(zip(self.row_id,self.col_id)):
            is_intersect,coord=self.isIntersect((i,j),self.intersec_coord)
            if is_intersect:
                self.intersect_length[coord]=idx
                
    def minSignalDelay(self,other_wire):
        return min(np.add(self.intersect_length, other_wire.intersect_length))
    
    def __str__(self):
        return str(self.sparse)

In [260]:
filename = os.path.join(os.getcwd(),"d3_input")
wires = []
with open(filename) as f:
    str_wires = f.read().splitlines()

In [261]:
def getMin(str_wires,size=100,center=None):
    wires = [Wire(wire,size=size,center=center) for wire in str_wires]
    
    while wires[0].flag_changed or wires[1].flag_changed:
        max_size = max(wires[0].size,wires[1].size)
        print(f"Common max size: {max_size}")
        wires = [Wire(wire,size=max_size,fixed_size=True) for wire in str_wires]

    return wires,wires[0].getClosestIntersectionDist(wires[1])

In [262]:
# Q1
str_wires = ["R8,U5,L5,D3","U7,R6,D4,L4"] # sol= 6, q2:30
# center = Point(8,1)
str_wires = ["R75,D30,R83,U83,L12,D49,R71,U7,L72","U62,R66,U55,R34,D71,R55,D58,R83"] # sol= 159, q2:610
center=None
str_wires = ["R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51","U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"] # sol= 135, q2:410
# str_wires = ["R4,U3,R1,D1,L6,U3,R2,D6,L2,D1,R4,U3,R4","U3,R3,U1,R2,D4,R1,U6,L4,D5,L4,D4,R3,U2,R3"]
# center = Point(7,3)

wires,sol=getMin(str_wires,10,center=center)
# print(wires[0])
print(sol)

Wire path created of size 318
Wire path created of size 301
Common max size: 318
Wire path created of size 477
Wire path created of size 477
Common max size: 477
Wire path created of size 477
Wire path created of size 477
135


In [263]:
#Q2
wires[0].signalAtIntersect()
wires[1].signalAtIntersect(other_wire=wires[0])
wires[0].minSignalDelay(wires[1])

410

# Day 4: Secure Container

In [40]:
from collections import Counter

In [41]:
d4_input = "264360-746325"
[min_val,max_val] = [int(val) for val in d4_input.split("-")]

In [64]:
def valid_code(value):
    str_val = str(value)
    prec_digit = str_val[0]
    for digit in str_val[1:]:
        if digit<prec_digit:
            return False
        prec_digit = digit

    c = Counter(str_val)
    if max(c.values())>=2:
        return True
    else:
        return False

In [65]:
# Simple test
code_test = [122345,111123,135679,111111,223450,123789]
res = [True,True,False,True,False,False]
for code,r in zip(code_test,res):
    print(valid_code(code)==r)

True
True
True
True
True
True


In [66]:
count=0
for val in range(min_val,max_val):
    if valid_code(val):
        count+=1
print(count)

945


In [67]:
# Part 2

def valid_code2(value):
    str_val = str(value)
    prec_digit = str_val[0]
    
    for digit in str_val[1:]:
        if digit<prec_digit:
            return False
        prec_digit = digit

    c = Counter(str_val)
    if 2 not in c.values():
        return False
    else:
        return True


In [68]:
# Simple test
code_test = [122345,111123,135679,111111,223450,123789,112233,123444,111122]
res = [True,False,False,False,False,False,True,False,True]
for code,r in zip(code_test,res):
    print(valid_code2(code)==r)

True
True
True
True
True
True
True
True
True


In [69]:
count=0
for val in range(min_val,max_val):
    if valid_code2(val):
        count+=1
print(count)

617
